In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

2024-06-01 20:41:10.422548: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-01 20:41:10.462706: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-01 20:41:11.146227: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
num_days = 365


start_date = datetime(2023, 1, 1)


dates = [start_date + timedelta(days=i) for i in range(num_days)]


sales = np.random.randint(10, 100, size=num_days)
sales1 = np.random.randint(9, 20, size=num_days)

df = pd.DataFrame({'data': dates, 'vendas': sales,'valor': sales})

df = df.reset_index().rename(columns={"index":"time_idx"})

df = df[['time_idx','vendas','valor']]

for col in df.select_dtypes(include=['int']):
    df[col] = df[col].astype(float)

df.head()

,time_idx,vendas,valor
0,0.0,88.0,88.0
1,1.0,71.0,71.0
2,2.0,98.0,98.0
3,3.0,61.0,61.0
4,4.0,37.0,37.0


In [3]:
dataset = tf.data.Dataset.from_tensor_slices(df)

2024-06-01 20:41:11.947581: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-01 20:41:11.975021: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-01 20:41:11.980500: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
for i in dataset.take(10):
    print(i)

tf.Tensor([ 0. 88. 88.], shape=(3,), dtype=float64)
tf.Tensor([ 1. 71. 71.], shape=(3,), dtype=float64)
tf.Tensor([ 2. 98. 98.], shape=(3,), dtype=float64)
tf.Tensor([ 3. 61. 61.], shape=(3,), dtype=float64)
tf.Tensor([ 4. 37. 37.], shape=(3,), dtype=float64)
tf.Tensor([ 5. 40. 40.], shape=(3,), dtype=float64)
tf.Tensor([ 6. 87. 87.], shape=(3,), dtype=float64)
tf.Tensor([ 7. 50. 50.], shape=(3,), dtype=float64)
tf.Tensor([ 8. 59. 59.], shape=(3,), dtype=float64)
tf.Tensor([ 9. 22. 22.], shape=(3,), dtype=float64)


2024-06-01 20:41:12.164066: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [5]:
for i in dataset.batch(5).take(2):
    print(i)

tf.Tensor(
[[ 0. 88. 88.]
 [ 1. 71. 71.]
 [ 2. 98. 98.]
 [ 3. 61. 61.]
 [ 4. 37. 37.]], shape=(5, 3), dtype=float64)
tf.Tensor(
[[ 5. 40. 40.]
 [ 6. 87. 87.]
 [ 7. 50. 50.]
 [ 8. 59. 59.]
 [ 9. 22. 22.]], shape=(5, 3), dtype=float64)


2024-06-01 20:41:12.912315: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [6]:
window_size = 7
batch_size = 30
shuffle_buffer =42

In [7]:
#def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
#dataset = tf.data.Dataset.from_tensor_slices(series)
#dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
#dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
#dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
#dataset = dataset.batch(batch_size).prefetch(1)
  #return dataset

In [8]:
dataset1 = dataset.window(window_size + 1, shift=1, drop_remainder=True)
for x in dataset1.take(5):
   for val in x:
      print(val.numpy(), end=' ')
   print()

[ 0. 88. 88.] [ 1. 71. 71.] [ 2. 98. 98.] [ 3. 61. 61.] [ 4. 37. 37.] [ 5. 40. 40.] [ 6. 87. 87.] [ 7. 50. 50.] 
[ 1. 71. 71.] [ 2. 98. 98.] [ 3. 61. 61.] [ 4. 37. 37.] [ 5. 40. 40.] [ 6. 87. 87.] [ 7. 50. 50.] [ 8. 59. 59.] 
[ 2. 98. 98.] [ 3. 61. 61.] [ 4. 37. 37.] [ 5. 40. 40.] [ 6. 87. 87.] [ 7. 50. 50.] [ 8. 59. 59.] [ 9. 22. 22.] 
[ 3. 61. 61.] [ 4. 37. 37.] [ 5. 40. 40.] [ 6. 87. 87.] [ 7. 50. 50.] [ 8. 59. 59.] [ 9. 22. 22.] [10. 46. 46.] 
[ 4. 37. 37.] [ 5. 40. 40.] [ 6. 87. 87.] [ 7. 50. 50.] [ 8. 59. 59.] [ 9. 22. 22.] [10. 46. 46.] [11. 29. 29.] 


2024-06-01 20:41:15.577600: W tensorflow/core/framework/dataset.cc:959] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.
2024-06-01 20:41:15.581323: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-01 20:41:15.585092: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-01 20:41:15.589164: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-01 20:41:15.591707: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-01 20:41:15.595755: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-01 20:41:15.596

In [9]:
dataset2 = dataset1.flat_map(lambda window: window.batch(window_size + 1))
for window in dataset2.take(3):
  print(window.numpy())

[[ 0. 88. 88.]
 [ 1. 71. 71.]
 [ 2. 98. 98.]
 [ 3. 61. 61.]
 [ 4. 37. 37.]
 [ 5. 40. 40.]
 [ 6. 87. 87.]
 [ 7. 50. 50.]]
[[ 1. 71. 71.]
 [ 2. 98. 98.]
 [ 3. 61. 61.]
 [ 4. 37. 37.]
 [ 5. 40. 40.]
 [ 6. 87. 87.]
 [ 7. 50. 50.]
 [ 8. 59. 59.]]
[[ 2. 98. 98.]
 [ 3. 61. 61.]
 [ 4. 37. 37.]
 [ 5. 40. 40.]
 [ 6. 87. 87.]
 [ 7. 50. 50.]
 [ 8. 59. 59.]
 [ 9. 22. 22.]]


2024-06-01 20:41:16.705182: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [14]:
dataset3 = dataset2.map(lambda window: (window[:,:1], window[:,1:]))

for x,y in dataset3:
    print("x = ", x.numpy())
    print("x shape = ", x.numpy().shape)
    print("y = ", y.numpy())
    print("y shape = ", y.numpy().shape)

x =  [[0.]
 [1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]]
x shape =  (8, 1)
y =  [[88. 88.]
 [71. 71.]
 [98. 98.]
 [61. 61.]
 [37. 37.]
 [40. 40.]
 [87. 87.]
 [50. 50.]]
y shape =  (8, 2)
x =  [[1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]
 [8.]]
x shape =  (8, 1)
y =  [[71. 71.]
 [98. 98.]
 [61. 61.]
 [37. 37.]
 [40. 40.]
 [87. 87.]
 [50. 50.]
 [59. 59.]]
y shape =  (8, 2)
x =  [[2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]
 [8.]
 [9.]]
x shape =  (8, 1)
y =  [[98. 98.]
 [61. 61.]
 [37. 37.]
 [40. 40.]
 [87. 87.]
 [50. 50.]
 [59. 59.]
 [22. 22.]]
y shape =  (8, 2)
x =  [[ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]]
x shape =  (8, 1)
y =  [[61. 61.]
 [37. 37.]
 [40. 40.]
 [87. 87.]
 [50. 50.]
 [59. 59.]
 [22. 22.]
 [46. 46.]]
y shape =  (8, 2)
x =  [[ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]
 [11.]]
x shape =  (8, 1)
y =  [[37. 37.]
 [40. 40.]
 [87. 87.]
 [50. 50.]
 [59. 59.]
 [22. 22.]
 [46. 46.]
 [29. 29.]]
y shape =  (8, 2)
x =  [[ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]
 [11.]
 [12.]]
x 

2024-06-01 20:42:07.174106: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [15]:
dataset4 = dataset3.shuffle(buffer_size=10)
for x,y in dataset4.take(2):
  print("x = ", x.numpy())
  print("x shape = ", x.numpy().shape)
  print("y = ", y.numpy())
  print("y shape = ", y.numpy().shape)

x =  [[ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]
 [11.]]
x shape =  (8, 1)
y =  [[37. 37.]
 [40. 40.]
 [87. 87.]
 [50. 50.]
 [59. 59.]
 [22. 22.]
 [46. 46.]
 [29. 29.]]
y shape =  (8, 2)
x =  [[ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]
 [11.]
 [12.]
 [13.]]
x shape =  (8, 1)
y =  [[87. 87.]
 [50. 50.]
 [59. 59.]
 [22. 22.]
 [46. 46.]
 [29. 29.]
 [16. 16.]
 [66. 66.]]
y shape =  (8, 2)


2024-06-01 20:42:12.012097: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [44]:

dataset5 = dataset4.batch(3).prefetch(1)
for x,y in dataset5:
    print("x = ", x.numpy())
    print("x shape = ", x.numpy().shape)
    print("y = ", y.numpy())
    print("y shape = ", y.numpy().shape)

x =  [[[ 8.]
  [ 9.]
  [10.]
  [11.]
  [12.]
  [13.]
  [14.]
  [15.]]

 [[ 4.]
  [ 5.]
  [ 6.]
  [ 7.]
  [ 8.]
  [ 9.]
  [10.]
  [11.]]

 [[ 0.]
  [ 1.]
  [ 2.]
  [ 3.]
  [ 4.]
  [ 5.]
  [ 6.]
  [ 7.]]]
x shape =  (3, 8, 1)
y =  [[[59. 59.]
  [22. 22.]
  [46. 46.]
  [29. 29.]
  [16. 16.]
  [66. 66.]
  [13. 13.]
  [30. 30.]]

 [[37. 37.]
  [40. 40.]
  [87. 87.]
  [50. 50.]
  [59. 59.]
  [22. 22.]
  [46. 46.]
  [29. 29.]]

 [[88. 88.]
  [71. 71.]
  [98. 98.]
  [61. 61.]
  [37. 37.]
  [40. 40.]
  [87. 87.]
  [50. 50.]]]
y shape =  (3, 8, 2)
x =  [[[ 9.]
  [10.]
  [11.]
  [12.]
  [13.]
  [14.]
  [15.]
  [16.]]

 [[ 7.]
  [ 8.]
  [ 9.]
  [10.]
  [11.]
  [12.]
  [13.]
  [14.]]

 [[ 1.]
  [ 2.]
  [ 3.]
  [ 4.]
  [ 5.]
  [ 6.]
  [ 7.]
  [ 8.]]]
x shape =  (3, 8, 1)
y =  [[[22. 22.]
  [46. 46.]
  [29. 29.]
  [16. 16.]
  [66. 66.]
  [13. 13.]
  [30. 30.]
  [65. 65.]]

 [[50. 50.]
  [59. 59.]
  [22. 22.]
  [46. 46.]
  [29. 29.]
  [16. 16.]
  [66. 66.]
  [13. 13.]]

 [[71. 71.]
  [98. 98.]
  [61. 

2024-06-01 21:02:41.266766: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [72]:
dataset5

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 2), dtype=tf.float64, name=None))>

In [81]:
for x,y in dataset5.as_numpy_iterator():
    print("x = ", x)
    print("x shape = ", x.shape)
    print("y = ", y)
    print("y shape = ", y.shape)

x =  [[[ 3.]
  [ 4.]
  [ 5.]
  [ 6.]
  [ 7.]
  [ 8.]
  [ 9.]
  [10.]]

 [[ 4.]
  [ 5.]
  [ 6.]
  [ 7.]
  [ 8.]
  [ 9.]
  [10.]
  [11.]]

 [[ 7.]
  [ 8.]
  [ 9.]
  [10.]
  [11.]
  [12.]
  [13.]
  [14.]]]
x shape =  (3, 8, 1)
y =  [[[61. 61.]
  [37. 37.]
  [40. 40.]
  [87. 87.]
  [50. 50.]
  [59. 59.]
  [22. 22.]
  [46. 46.]]

 [[37. 37.]
  [40. 40.]
  [87. 87.]
  [50. 50.]
  [59. 59.]
  [22. 22.]
  [46. 46.]
  [29. 29.]]

 [[50. 50.]
  [59. 59.]
  [22. 22.]
  [46. 46.]
  [29. 29.]
  [16. 16.]
  [66. 66.]
  [13. 13.]]]
y shape =  (3, 8, 2)
x =  [[[ 5.]
  [ 6.]
  [ 7.]
  [ 8.]
  [ 9.]
  [10.]
  [11.]
  [12.]]

 [[11.]
  [12.]
  [13.]
  [14.]
  [15.]
  [16.]
  [17.]
  [18.]]

 [[10.]
  [11.]
  [12.]
  [13.]
  [14.]
  [15.]
  [16.]
  [17.]]]
x shape =  (3, 8, 1)
y =  [[[40. 40.]
  [87. 87.]
  [50. 50.]
  [59. 59.]
  [22. 22.]
  [46. 46.]
  [29. 29.]
  [16. 16.]]

 [[29. 29.]
  [16. 16.]
  [66. 66.]
  [13. 13.]
  [30. 30.]
  [65. 65.]
  [35. 35.]
  [91. 91.]]

 [[46. 46.]
  [29. 29.]
  [16. 

2024-06-01 21:34:27.250382: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
np.zeros(shape=(dataset5[0]))

In [64]:
encoder_input = np.array([])
decoder_input = np.array([])

np.app

In [65]:
for window in dataset5:
    for x,y in window:
        for batch_x in x:
            np.append(encoder_input,batch_x,axis=0)
        for batch_y in y:
            np.append(decoder_input,batch_y,axis=0)
        

ValueError: too many values to unpack (expected 2)

In [63]:
list(dataset5)[1][0][1]

2024-06-01 21:19:08.527137: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


<tf.Tensor: shape=(8, 1), dtype=float64, numpy=
array([[2.],
       [3.],
       [4.],
       [5.],
       [6.],
       [7.],
       [8.],
       [9.]])>

In [41]:
data_input = dataset5

# Novos testes

In [83]:
dataset_encoder = tf.data.Dataset.from_tensor_slices(df.time_idx)
dataset_decoder = tf.data.Dataset.from_tensor_slices(df.iloc[:,1:])

In [88]:
def create_batches(encoder_df,decoder_df,batch_size):
    dataset_encoder = tf.data.Dataset.from_tensor_slices(encoder_df)
    dataset_decoder = tf.data.Dataset.from_tensor_slices(decoder_df)

    batched_dataset_encoder = dataset_encoder.batch(batch_size).prefetch(1)
    batched_dataset_decoder = dataset_decoder.batch(batch_size).prefetch(1)

    return batched_dataset_encoder,batched_dataset_decoder

dataset_encoder, dataset_decoder = create_batches(df.time_idx,df.iloc[:,1:],batch_size=7)



In [99]:
dataset = tf.data.Dataset.from_tensor_slices((df.time_idx, df.iloc[:,1:], df.iloc[:,1:]))

# Optional padding for variable length sequences
padded_dataset = dataset.padded_batch(
    batch_size=32#,
    #padded_shapes=([encoder_max_length], [decoder_max_length], [decoder_max_length]),
    #padding_values=(0, 0, 0)
)

# Optional shuffling for better training
shuffled_dataset = padded_dataset.shuffle(buffer_size=len(dataset))

In [100]:
shuffled_dataset

<_ShuffleDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.float64, name=None), TensorSpec(shape=(None, 2), dtype=tf.float64, name=None), TensorSpec(shape=(None, 2), dtype=tf.float64, name=None))>

In [95]:
class Encoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_untis:int):
        super(Encoder_layer,self).__init__()


        self.LSTM_encoder = tf.keras.layers.LSTM(units=num_untis,
                                                 return_state = True,
                                                 name = 'LSTM_encoder')
        
    def call(self,inputs):
        
        output_encoder, state_h, state_c = self.LSTM_encoder(inputs)

        return output_encoder, state_h, state_c
    

class Decoder_layer(tf.keras.layers.Layer):
    def __init__(self, num_untis:int):
        super(Decoder_layer, self).__init__()

        self.LSTM_decoder = tf.keras.layers.LSTM(units=num_untis,
                                                 return_sequences=True,
                                                 return_state = True,
                                                 name = 'LSTM_decoder')

        self.Dense_output = tf.keras.layers.Dense(2)
    def call(self,inputs,encoder_states):


        output_decoder, state_h, state_c = self.LSTM_decoder(inputs,initial_state=encoder_states)
        output = self.Dense_output(output_decoder)

        return output
    
class Seq_to_Seq_LSTM(tf.keras.Model):
    def __init__(self, num_units:int):

        super().__init__()
        self.num_units = num_units


        self.encoder_layer = Encoder_layer(self.num_units)
        self.decoder_layer = Decoder_layer(self.num_units)


    def call(self,inputs):

        enc,dec= inputs
       
        _, state_h, state_c = self.encoder_layer(enc)

        output_decoder = self.decoder_layer(dec, encoder_states = [state_h, state_c])

        return output_decoder

In [101]:

model = Seq_to_Seq_LSTM(num_units=64)

model.compile(optimizer='rmsprop', loss='mae')


model.fit(shuffled_dataset,epochs=100,verbose=False)

OperatorNotAllowedInGraphError: Exception encountered when calling Seq_to_Seq_LSTM.call().

[1mIterating over a symbolic `tf.Tensor` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.[0m

Arguments received by Seq_to_Seq_LSTM.call():
  • inputs=tf.Tensor(shape=(None,), dtype=float32)